<a href="https://colab.research.google.com/github/bhargav-joshi/DataWareHousing-Mining/blob/main/Mushroom_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/coding-blocks-archives/DS-Live-Jun-20/master/session%2015/mushrooms.csv")

In [3]:
df.shape

(8124, 23)

In [4]:
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


### Encode Categorical data into numbers

In [5]:
le = LabelEncoder()

In [6]:
df = df.apply(le.fit_transform)

In [7]:
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


In [8]:
data = df.values

In [9]:
data[:, 0]

array([1, 0, 0, ..., 0, 1, 0])

In [10]:
X = data[:, 1:]
y = data[:, 0]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Classifier

In [13]:
def prior_prob(y_train, label):
    total_examples = y_train.shape[0]
    classes_examples = np.sum(y_train == label)
    
    return (classes_examples)/float(total_examples)

In [14]:
prior_prob(y_train, 1)

0.48222803508232037

In [15]:
prior_prob(y_train , 0)

0.5177719649176796

In [16]:
a = np.array([1, 0,0,0,0,1,1,0,0,0 ])
prior_prob(a, 1)

0.3

In [17]:
def cond_prob(X_train, y_train, feature_col, feature_val, label):
    X_filtered = X_train[y_train==label]
    
    numerator = np.sum(X_filtered[:, feature_col] == feature_val)
    denom = np.sum(y_train==label)
    
    return numerator/float(denom)

## Compute : Likelihood and Posterior Probability

In [18]:
def predict(X_train, y_train, x_query):
    "x_query is a single testing point having n features"

    classes = np.unique(y_train)
    n_features = X_train.shape[1]
    post_prob = []
    
    for label in classes:
        likelihood = 1.0
        # likelihood
        for f in range(n_features):
            cond = cond_prob(X_train, y_train, f, x_query[f], label)
            likelihood = likelihood*cond
            
        
        # prior
        prior = prior_prob(y_train, label)
        post = likelihood*prior
        
        post_prob.append(post)
        
    pred = np.argmax(post_prob)
    return pred

In [19]:
predict(X_train, y_train, X_test[79])

1

In [20]:
y_test[79]

1

In [21]:
X_test.shape

(1625, 22)

In [22]:
def accuracy(X_train, y_train, X_test, y_test):
    y_pred = []
    
    for i in range(X_test.shape[0]):
        pred_label = predict(X_train, y_train, X_test[i])
        y_pred.append(pred_label)
        
    y_pred = np.array(y_pred)
    
    acc = np.sum(y_pred == y_test)/ y_test.shape[0]
    
    return acc

In [23]:
accuracy(X_train, y_train, X_test, y_test)

0.9963076923076923

# Gaussian Naive Bayes

In [24]:
from sklearn.datasets import load_iris 
iris = load_iris() 

In [25]:
X = iris.data
y = iris.target

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

In [27]:
from sklearn.naive_bayes import GaussianNB

In [28]:
gnb = GaussianNB()

In [29]:
gnb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [30]:
gnb.predict(X_test)

array([0, 1, 1, 0, 2, 2, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2,
       1, 2, 0, 0, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 1])

In [31]:
gnb.score(X_test, y_test)

0.95

In [32]:
X_test[0]

array([5.8, 4. , 1.2, 0.2])